# Introduction

In a [previous blog post](https://drchrislevy.github.io/blog), I explored various aspects of building AI agents, including coding agents, ReAct prompting, and tool-calling loops. Recently, OpenAI announced [new tools for building agents](https://openai.com/index/new-tools-for-building-agents/), introducing the Responses API and Agents SDK. OpenAI set the standard with their Chat Completions API, which has been widely adopted and extended by developers.

However, it's important to remain cautious. Remember what happened to the Assistants API? It never fully emerged from beta and is scheduled to be sunset in 2026. In contrast, these new APIs are not marked as beta, indicating that OpenAI feels more confident about this direction. Given their prior success with the Chat Completions API, there's reason to be optimistic that they might achieve similar success again. OpenAI has likely incorporated key learnings from their experience with the Assistants API into these latest developments, making these new tools intriguing and well worth exploring further.

# Responses API

OpenAI plans to continue supporting the [Chat Completions API](https://platform.openai.com/docs/guides/responses-vs-chat-completions#the-chat-completions-api-is-not-going-away). However, for new projects, they recommend using the newly introduced [Responses API](https://platform.openai.com/docs/api-reference/responses). 

One advantage of the Chat Completions API I've appreciated is its wide adoption by other LLM providers, making it easy to switch between services. Because of this flexibility, it may still be practical to use Chat Completions for some new projects. It remains to be seen whether other providers will adopt the Responses API as well.

Here's a screenshot from the OpenAI documentation explaining the Responses API:

<img src="static_blog_imgs/why_responses_api.png" width="75%">

### Key points to note about the Responses API:

- **Stateful**: It includes a `previous_response_id` to support long-running conversations.
- It is distinctly different from the Chat Completions API.
- If your application doesn't require built-in tools, you can confidently continue using **Chat Completions**.
- When ready for advanced capabilities tailored to agent workflows, the Responses API is recommended.
- The Responses API represents OpenAI's future direction for agent-building.


## Quickstart

I'm not going to go over all the details of the Responses API, because that's what the docs are for.
But I'm going to cover some things that are **new** to me.


In [1]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client = OpenAI()

response = client.responses.create(model="gpt-4o-mini", input="Tell a quick dad joke!")

print(response)

Response(id='resp_67dcb9129b288191b03676b4c3ee91770b18a606f20fdf95', created_at=1742518546.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseOutputMessage(id='msg_67dcb912f2f48191911626bf906ad6840b18a606f20fdf95', content=[ResponseOutputText(annotations=[], text="Why don't skeletons fight each other? \n\nThey don't have the guts!", type='output_text')], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reasoning(effort=None, generate_summary=None), status='completed', text=ResponseTextConfig(format=ResponseFormatText(type='text')), truncation='disabled', usage=ResponseUsage(input_tokens=31, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=16, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=47), user=

In [2]:
response.to_dict()

{'id': 'resp_67dcb9129b288191b03676b4c3ee91770b18a606f20fdf95',
 'created_at': 1742518546.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'response',
 'output': [{'id': 'msg_67dcb912f2f48191911626bf906ad6840b18a606f20fdf95',
   'content': [{'annotations': [],
     'text': "Why don't skeletons fight each other? \n\nThey don't have the guts!",
     'type': 'output_text'}],
   'role': 'assistant',
   'status': 'completed',
   'type': 'message'}],
 'parallel_tool_calls': True,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [],
 'top_p': 1.0,
 'max_output_tokens': None,
 'previous_response_id': None,
 'reasoning': {'effort': None, 'generate_summary': None},
 'status': 'completed',
 'text': {'format': {'type': 'text'}},
 'truncation': 'disabled',
 'usage': {'input_tokens': 31,
  'input_tokens_details': {'cached_tokens': 0},
  'output_tokens': 16,
  'output_tokens_details': {'reasoning_tokens': 0},


In [3]:
print(response.output[0].content[0].text)

Why don't skeletons fight each other? 

They don't have the guts!


Or a little shortcut:

In [4]:
print(response.output_text)

Why don't skeletons fight each other? 

They don't have the guts!


- Response objects are saved for 30 days by default. You can disable this behavior by setting store to false when creating a Response.
- Can be viewed in the dashboard logs page or retrieved via the API.

I never inspected traces in chat completions before because I don't think they are enabled by default.
But here with Responses API, you can inspect traces by default for 30 days unless you disable it on the API call.

<img src="static_blog_imgs/responses_logs.png" width="100%">

<img src="static_blog_imgs/responses_log_example.png" width="100%">

Or retrieve traces via the API.

In [5]:
client.responses.retrieve(response.id)

Response(id='resp_67dcb9129b288191b03676b4c3ee91770b18a606f20fdf95', created_at=1742518546.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseOutputMessage(id='msg_67dcb912f2f48191911626bf906ad6840b18a606f20fdf95', content=[ResponseOutputText(annotations=[], text="Why don't skeletons fight each other? \n\nThey don't have the guts!", type='output_text')], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reasoning(effort=None, generate_summary=None), status='completed', text=ResponseTextConfig(format=ResponseFormatText(type='text')), truncation='disabled', usage=ResponseUsage(input_tokens=31, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=16, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=47), user=

## Instruction Following

In [6]:
response = client.responses.create(model="gpt-4o-mini", instructions="You return markdown and lots of emojis. ", input="Tell a quick dad joke!")
print(response.output_text)

Why did the scarecrow win an award? 🌾🏆 

Because he was outstanding in his field! 😄


*The instructions parameter gives the model high-level instructions on how it should behave while generating a response, including tone, goals, and examples of correct responses. Any instructions provided this way will take priority over a prompt in the input parameter.* [source](https://platform.openai.com/docs/guides/text#message-roles-and-instruction-following)

This example is roughly equivalent to:

In [7]:
response = client.responses.create(
    model="gpt-4o-mini",
    input=[{"role": "developer", "content": "You return markdown and lots of emojis. "}, {"role": "user", "content": "Tell a quick dad joke!"}],
)

print(response.output_text)

Why did the scarecrow win an award? 🎉

Because he was outstanding in his field! 🌾😂


The argument `instructions` is used to insert a system (or developer) message as the first item in the model's context [source](https://platform.openai.com/docs/api-reference/responses/create#responses-create-instructions).






In [8]:
response = client.responses.create(
    model="gpt-4o-mini",
    input=[{"role": "developer", "content": "You return markdown and lots of emojis. "}, {"role": "user", "content": "Tell a quick dad joke!"}],
    instructions="You must talk like a pirate and do not return any markdown or emojis.",
)

print(response.output_text)

Why did the scarecrow win an award? Because he was outstanding in his field!


<img src="static_blog_imgs/instructions_insertion.png" width="100%">

## Conversation State

We can manually handle the chat history using alternating `user` and `assistant` messages, just as previously done with the chat completions API.



In [9]:
response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role": "developer", "content": "You are a helpful assistant."},
        {"role": "user", "content": "My name is Chris, and my age is 40."},
        {"role": "assistant", "content": "Nice to meet you, Chris!"},
        {"role": "user", "content": "How old am I?"},
    ],
)

print(response.output_text)

You mentioned that you are 40 years old.


Alternately, we can use the `previous_response_id` parameter to manage conversation state.

In [10]:
response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role": "developer", "content": "You are a helpful assistant."},
        {"role": "user", "content": "My name is Chris, and my age is 40."},
    ],
)

print(response.output_text)
print(response.id)

Nice to meet you, Chris! If there's anything specific you'd like to talk about or any questions you have, feel free to let me know!
resp_67dcb9189d248191aa8b5380deffe0170a7498d0c4209037


In [11]:
response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role": "user", "content": "How old am I"},
    ],
    previous_response_id=response.id,
)
print(response.output_text)
print(response.id)

You mentioned that you are 40 years old.
resp_67dcb919f984819194f8ba91b5c7d4130a7498d0c4209037


In [12]:
response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role": "user", "content": "And what was my name?"},
    ],
    previous_response_id=response.id,
)
print(response.output_text)
print(response.id)

Your name is Chris.
resp_67dcb91ae9148191a8974cf8d267ba5e0a7498d0c4209037


When using `previous_response_id`, all previous input tokens for responses in the chain are billed as input tokens in the API [source](https://platform.openai.com/docs/guides/conversation-state#openai-apis-for-conversation-state).

When you view the logs in dashboard for a message that used `previous_response_id`, there is a link/button to find the previous response.

<img src="static_blog_imgs/prev_response_log.png" width="100%">


When using `previous_response_id`, the **instructions** from a previous response will be not be carried over to the next response. This makes it simple to swap out system (or developer) messages in new responses [source](https://platform.openai.com/docs/api-reference/responses/create#responses-create-instructions). The instructions parameter only applies to the current response generation request.

In [13]:
response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role": "user", "content": "What country is the city Toronto in?"},
    ],
    instructions="You only write lower case letters",
)
print(response.output_text)
response.id

toronto is in canada.


'resp_67dcb91bd80c8191b96a2e4c3ed03b3508e4210ba7e03814'

In [14]:
response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role": "user", "content": "What country was it again?"},
    ],
    previous_response_id=response.id,
)
print(response.output_text)

Toronto is in Canada.


In [15]:
response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role": "user", "content": "I forget, what was it?"},
    ],
    instructions="You only write UPPER CASE letters",
    previous_response_id=response.id,
)
print(response.output_text)

TORONTO IS IN CANADA.


Of course there all the other OpenAI LLM goodies such as function calling, structured outputs, streaming, analyzing images, and so on.
They even released a bunch of new audio features (see [here](https://openai.com/index/introducing-our-next-generation-audio-models/()) and [here](https://platform.openai.com/docs/guides/audio()))
I'm just going to cover some thins that are new to me, as a way to get familiar with some of the new features.

## Built-in tools

Something new here to me is the ability to use built-in tools with the Responses API.
As of writing, the built-in tools include things like web search, file search, computer use, and function calling.
I'm already familiar with tool/function calling. But let's take a look at some of these other tools.







In [13]:
# Importing these to make the output look nicer
from fasthtml.common import show
from monsterui.all import render_md

/Users/christopher/personal_projects/DrChrisLevy.github.io/.venv/lib/python3.11/site-packages/soupsieve/css_match.py:807: RuntimeWarning: coroutine 'Runner.run' was never awaited
  def match_future_relations(self, el: bs4.Tag, relation: ct.SelectorList) -> bool:


### Web Search

In [25]:
response = client.responses.create(
    model="gpt-4o-mini",
    tools=[{"type": "web_search_preview"}],
    input="Did Alabama State win their first game of the March Madness tournament in 2025?",  # web_search_preview_2025_03_11 points to a dated version of the tool
)

show(render_md(response.output_text))

In [26]:
response.to_dict()

{'id': 'resp_67dcbc296a848191beea151d745164f5086920f67eda0861',
 'created_at': 1742519337.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'response',
 'output': [{'id': 'ws_67dcbc2a09ec8191991c57849138d81d086920f67eda0861',
   'status': 'completed',
   'type': 'web_search_call'},
  {'id': 'msg_67dcbc2c6eac8191a9bd11350095c2b8086920f67eda0861',
   'content': [{'annotations': [{'end_index': 363,
       'start_index': 210,
       'title': 'Alabama State stuns Saint Francis for last-second First Four victory',
       'type': 'url_citation',
       'url': 'https://www.reuters.com/sports/basketball/alabama-state-stuns-saint-francis-last-second-first-four-victory-2025-03-19/?utm_source=openai'},
      {'end_index': 627,
       'start_index': 463,
       'title': 'No. 1 overall seed Auburn puts away Alabama State 83-63 to open March Madness',
       'type': 'url_citation',
       'url': 'https://www.clickorl

You can also force the use of the web_search_preview tool by using the tool_choice parameter, and setting it to `{type: "web_search_preview"}`:


In [27]:
response = client.responses.create(
    model="gpt-4o-mini",
    tools=[{"type": "web_search_preview"}],
    tool_choice={"type": "web_search_preview"},
    input="Is there an upcoming federal election in Canada?",
)
show(render_md(response.output_text))

In [28]:
response.to_dict()

{'id': 'resp_67dcbc2fa508819182c54a57f44f00fb00684d0dd54f2bf5',
 'created_at': 1742519343.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'response',
 'output': [{'id': 'ws_67dcbc2fb01c8191aaa188f79e91d28100684d0dd54f2bf5',
   'status': 'completed',
   'type': 'web_search_call'},
  {'id': 'msg_67dcbc3211888191b2b8fc4c36c1899000684d0dd54f2bf5',
   'content': [{'annotations': [{'end_index': 313,
       'start_index': 209,
       'title': '45th Canadian federal election - Wikipedia',
       'type': 'url_citation',
       'url': 'https://en.m.wikipedia.org/wiki/45th_Canadian_federal_election?utm_source=openai'},
      {'end_index': 894,
       'start_index': 736,
       'title': 'Canada PM Carney to call for April 28 election on Sunday, Globe and Mail reports',
       'type': 'url_citation',
       'url': 'https://www.reuters.com/world/americas/canada-pm-carney-expected-call-snap-election-april-28-globe-

Note that for a web search we have:

- A `web_search_call` output item with the ID of the search call.

In [29]:
response.output[0]

ResponseFunctionWebSearch(id='ws_67dcbc2fb01c8191aaa188f79e91d28100684d0dd54f2bf5', status='completed', type='web_search_call')

- the annotations:




In [30]:
response.output[1].content[0].annotations

[AnnotationURLCitation(end_index=313, start_index=209, title='45th Canadian federal election - Wikipedia', type='url_citation', url='https://en.m.wikipedia.org/wiki/45th_Canadian_federal_election?utm_source=openai'),
 AnnotationURLCitation(end_index=894, start_index=736, title='Canada PM Carney to call for April 28 election on Sunday, Globe and Mail reports', type='url_citation', url='https://www.reuters.com/world/americas/canada-pm-carney-expected-call-snap-election-april-28-globe-mail-reports-2025-03-20/?utm_source=openai'),
 AnnotationURLCitation(end_index=1163, start_index=938, title='Canada PM Carney to call for April 28 election on Sunday, Globe and Mail reports', type='url_citation', url='https://www.reuters.com/world/americas/canada-pm-carney-expected-call-snap-election-april-28-globe-mail-reports-2025-03-20/?utm_source=openai'),
 AnnotationURLCitation(end_index=1298, start_index=1166, title='Canada election expected to be held on April 28', type='url_citation', url='https://ww

- text content:

In [31]:
response.output[1].content[0].text

"Yes, there is an upcoming federal election in Canada. The 45th Canadian federal election is scheduled to take place on or before October 20, 2025, as per the fixed-date provisions of the Canada Elections Act. ([en.m.wikipedia.org](https://en.m.wikipedia.org/wiki/45th_Canadian_federal_election?utm_source=openai))\n\nHowever, recent reports indicate that Prime Minister Mark Carney plans to call for an early election on April 28, 2025. Carney, who became Prime Minister after Justin Trudeau's resignation, aims to secure a strong mandate amid ongoing trade tensions and sovereignty threats from U.S. President Donald Trump. The election would see Carney's Liberal Party facing off against the Conservative Party led by Pierre Poilievre. ([reuters.com](https://www.reuters.com/world/americas/canada-pm-carney-expected-call-snap-election-april-28-globe-mail-reports-2025-03-20/?utm_source=openai))\n\n\n## Canada's Upcoming Federal Election:\n- [Canada PM Carney to call for April 28 election on Sund

To refine search results based on geography, you can specify an approximate user location using country, city, region, and/or timezone.




In [32]:
response = client.responses.create(
    model="gpt-4o-mini",
    tools=[
        {
            "type": "web_search_preview",
            "user_location": {
                "type": "approximate",
                "country": "CA",  #  two-letter ISO country code
                "city": "Halifax",  # free text strings
                "region": "Nova Scotia",  # free text strings
            },
        }
    ],
    input="What are the best restaurants around Halifax?",
)

show(render_md(response.output_text))

The parameter `search_context_size` controls the number of search results. The tokens used by the search tool **do not** affect the context window of the main model.
Choosing the `search_context_size` parameter is a trade-off between cost, quality, and latency. The available values are 'high', 'medium', and 'low'. The default is 'medium'.
The [pricing page](https://platform.openai.com/docs/pricing) has all the details

In [34]:
response = client.responses.create(
    model="gpt-4o",
    tools=[
        {
            "type": "web_search_preview",
            "user_location": {
                "type": "approximate",
            },
            "search_context_size": "high",
        }
    ],
    input="Give me a markdown table of all the 2025 March Madness games and scores that have been played so far.",
)

show(render_md(response.output_text))

Date,Round,Matchup,Score
March 18,First Four,(16) Alabama State vs. (16) Saint Francis,70-68
March 18,First Four,(11) North Carolina vs. (11) San Diego State,95-68
March 19,First Four,(16) Mount St. Mary's vs. (16) American,83-72
March 19,First Four,(11) Xavier vs. (11) Texas,86-80
March 20,First Round,(9) Creighton vs. (8) Louisville,89-75
March 20,First Round,(4) Purdue vs. (13) High Point,75-63
March 20,First Round,(3) Wisconsin vs. (14) Montana,85-66
March 20,First Round,(1) Houston vs. (16) SIU Edwardsville,78-40
March 20,First Round,(1) Auburn vs. (16) Alabama State,83-63
March 20,First Round,(12) McNeese vs. (5) Clemson,69-67


## File Search

File search in the Responses API allows models to easily access information from previously uploaded files using semantic and keyword search. By uploading files to vector stores, you enhance the model's built-in knowledge without additional coding—OpenAI manages everything automatically. It's essentially RAG (Retrieval-Augmented Generation) hosted by OpenAI. The documentation is [here](https://platform.openai.com/docs/guides/tools-file-search).

**TODO:** Come back to this when interested.

## Computer Use

**TODO:** Come back when interested. See [here](https://platform.openai.com/docs/guides/tools-computer-use).

# Agent SDK

This is what I came here to learn more about.
I already wrote about [agents part 1](https://drchrislevy.github.io/blog) extensively in a previous blog post, but I never used any frameworks.
I built from scratch to understand the concepts.

There are a lot of pieces here that can be glued together, as seen from the OpenAI documentation [here](https://platform.openai.com/docs/guides/agents).
Here is an image from their documentation showing the components:

<img src="static_blog_imgs/openai_agent_components.png" width="75%">

<img src="static_blog_imgs/model_use_cases.png" width="75%">





## Hello World

When you read the [docs](https://openai.github.io/openai-agents-python/#hello-world-example)
and try to run the opening example (which uses `run_sync`), it won't work from a notebook. `run_sync` is a convenience method that wraps async code to run synchronously by creating a new event loop. It's meant for simple synchronous scripts. However, this causes problems in environments that already have an event loop running, such as Jupyter notebooks, etc.  So we will use `run` with `await` directly at the notebook level when required.

In [52]:
from agents import Agent, Runner

agent = Agent(name="Chris", instructions="You are a helpful assistant.")
result = await Runner.run(agent, "Hey")
print(result.final_output)

Hello! How can I assist you today?


In [54]:
result.to_input_list()

[{'content': 'Hey', 'role': 'user'},
 {'id': 'msg_67e82342da08819182c3d6371ede384007864a236e990531',
  'content': [{'annotations': [],
    'text': 'Hello! How can I assist you today?',
    'type': 'output_text'}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'}]

In [59]:
result

RunResult(input='Hey', new_items=[MessageOutputItem(agent=Agent(name='Chris', instructions='You are a helpful assistant.', handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[], mcp_servers=[], input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), raw_item=ResponseOutputMessage(id='msg_67e82342da08819182c3d6371ede384007864a236e990531', content=[ResponseOutputText(annotations=[], text='Hello! How can I assist you today?', type='output_text')], role='assistant', status='completed', type='message'), type='message_output_item')], raw_responses=[ModelResponse(output=[ResponseOutputMessage(id='msg_67e82342da08819182c3d6371ede384007864a236e990531', content=[ResponseOutputText(annotations=[], text='Hello! How can I assist you t

## Building an Agent with Coding Abilities

In [Part 1](https://drchrislevy.github.io//blog), I implemented a CodeACT-style agent from scratch.  
This agent was able to express all of its actions directly in code, rather than relying on JSON-style tools.

Now that we're using the OpenAI Agent SDK, we're limited to JSON-style tools, so we can't use the CodeACT approach directly.  
I actually created an [issue on the OpenAI Agents SDK GitHub repo](https://github.com/openai/openai-agents-python/issues/383) to ask whether there are any future plans to support CodeACT-style agents within the SDK.  
I think it would be a great addition, though it would likely require a fair amount of work to implement. There are no known plans at the time of writing this.

Anyway, back to the task at hand. Let's build an agent using the OpenAI Agent SDK that can write arbitrary Python code to solve a task.  
We'll execute that code in a sandboxed environment using Modal. I also covered this in [Part 1](https://drchrislevy.github.io//blog) of the blog post.

Here is the code which uses Modal to execute the code in a sandboxed environment:


In [210]:
import json

import modal

# Create image with IPython installed
image = modal.Image.debian_slim().pip_install("ipython", "pandas")


# Create the driver program that will run in the sandbox
def create_driver_program():
    return """
import json
import sys
import re
from IPython.core.interactiveshell import InteractiveShell
from IPython.utils.io import capture_output

def strip_ansi_codes(text):
    ansi_escape = re.compile(r'\\x1B(?:[@-Z\\\\-_]|\\[[0-?]*[ -/]*[@-~])')
    return ansi_escape.sub('', text)

# Create a persistent IPython shell instance
shell = InteractiveShell()
shell.colors = 'NoColor'  # Disable color output
shell.autoindent = False  # Disable autoindent

# Keep reading commands from stdin
while True:
    try:
        # Read a line of JSON from stdin
        command = json.loads(input())
        code = command.get('code')
        
        if code is None:
            print(json.dumps({"error": "No code provided"}))
            continue
            
        # Execute the code and capture output
        with capture_output() as captured:
            result = shell.run_cell(code)

        # Clean the outputs
        stdout = strip_ansi_codes(captured.stdout)
        stderr = strip_ansi_codes(captured.stderr)
        error = strip_ansi_codes(str(result.error_in_exec)) if not result.success else None

        # Format the response
        response = {
            "stdout": stdout,
            "stderr": stderr,
            "success": result.success,
            "result": repr(result.result) if result.success else None,
            "error": error
        }
        
        # Send the response
        print(json.dumps(response), flush=True)
        
    except Exception as e:
        print(json.dumps({"error": strip_ansi_codes(str(e))}), flush=True)
"""


def create_sandbox():
    """Creates and returns a Modal sandbox running an IPython shell."""
    app = modal.App.lookup("ipython-sandbox", create_if_missing=True)

    # Create the sandbox with the driver program
    with modal.enable_output():
        sandbox = modal.Sandbox.create("python", "-c", create_driver_program(), image=image, app=app)

    return sandbox


def execute_python_code(code: str, sandbox=None) -> dict:
    created_sandbox = False
    if sandbox is None:
        sandbox = create_sandbox()
        created_sandbox = True
    # Send the code to the sandbox
    sandbox.stdin.write(json.dumps({"code": code}))
    sandbox.stdin.write("\n")
    sandbox.stdin.drain()

    # Get the response
    response = next(iter(sandbox.stdout))
    if created_sandbox:
        sandbox.terminate()
    return json.loads(response)

I will make it a `tool` that can be used by the agent.
We will also add the `WebSearchTool` to the agent as well.

In [219]:
from agents import WebSearchTool, function_tool

sb = create_sandbox()

@function_tool()
def execute_code(code: str) -> dict:
    """Execute the given Python code in a sandboxed environment and return the output."""
    return execute_python_code(code, sb)

In [220]:
agent = Agent(
    name="Code Agent",
    instructions="""
You solve tasks using an agentic coding loop in Python.

Follow this loop carefully:

- Think and write code. Send the code to the execute_code tool.
- Get code output results back.
- Think again, improve or add more code.
- Get code output results back.
...
- Repeat until you've solved the problem completely.

If you encounter any errors:

- FIX THEM and continue the loop.
- If modules are not found, install them using: !pip install <package_name>

Never give up. Continue iterating until the task is fully solved.
The sandbox environment is safe, isolated, and supports running arbitrary Python code.
State is maintained between code snippets:
Variables and definitions persist across executions.

Example:

```python
# first code snippet
x = 2
print(x)
```

```python
# second code snippet
y = 6
print(x + y)  # This works because state persist
```

Begin your agentic coding loop!

""",
    model="gpt-4o",
    tools=[execute_code, WebSearchTool()],
)



In [221]:
result = await Runner.run(agent, "Use an open source weather API to get the temperature in Halifax, Nova Scotia for the last 60 days. Then fit a statistical model to predict the temperature for the next week. Print the final predictions in a markdown table. Todays date is 2025-04-01.", max_turns=20)
print(result.final_output)

We've successfully fit the ARIMA model and forecasted the next 7 days' temperatures. Here are the predictions presented in a markdown table:

| Date       | Predicted Temperature |
|------------|-----------------------|
| 2025-04-01 | 6.14°C                |
| 2025-04-02 | 6.15°C                |
| 2025-04-03 | 6.15°C                |
| 2025-04-04 | 6.15°C                |
| 2025-04-05 | 6.15°C                |
| 2025-04-06 | 6.15°C                |
| 2025-04-07 | 6.15°C                |

If you need further analysis or different models, feel free to ask!


There are some different attributes that can be accessed on the result object. Here are some others worth noting:

In [311]:
result.input

'Use an open source weather API to get the temperature in Halifax, Nova Scotia for the last 60 days. Then fit a statistical model to predict the temperature for the next week. Print the final predictions in a markdown table. Todays date is 2025-04-01.'

In [312]:
result.raw_responses

[ModelResponse(output=[ResponseOutputMessage(id='msg_67ec8043d1f88191ab7a95ab93ccdc4508ede4ef04aa1260', content=[ResponseOutputText(annotations=[], text="To solve this problem, we'll follow these steps:\n\n1. **Find an Open Source Weather API**: Identify an API to retrieve past weather data for Halifax, Nova Scotia.\n2. **Fetch the Data**: Gather temperature data for the past 60 days.\n3. **Process Data**: Preprocess the data to fit a statistical model.\n4. **Fit a Model**: Use a statistical model to predict the next week's temperatures.\n5. **Output Predictions**: Print the predictions in a markdown table.\n\nLet's start by finding and using an open source weather API. I'll search for one first.", type='output_text')], role='assistant', status='completed', type='message'), ResponseFunctionWebSearch(id='ws_67ec80467e4c819181200d2eecb7d08608ede4ef04aa1260', status='completed', type='web_search_call'), ResponseOutputMessage(id='msg_67ec804887488191a7cde17165e9b72b08ede4ef04aa1260', conte

In [313]:
result.new_items

[MessageOutputItem(agent=Agent(name='Code Agent', instructions="\nYou solve tasks using an agentic coding loop in Python.\n\nFollow this loop carefully:\n\n- Think and write code. Send the code to the execute_code tool.\n- Get code output results back.\n- Think again, improve or add more code.\n- Get code output results back.\n...\n- Repeat until you've solved the problem completely.\n\nIf you encounter any errors:\n\n- FIX THEM and continue the loop.\n- If modules are not found, install them using: !pip install <package_name>\n\nNever give up. Continue iterating until the task is fully solved.\nThe sandbox environment is safe, isolated, and supports running arbitrary Python code.\nState is maintained between code snippets:\nVariables and definitions persist across executions.\n\nExample:\n\n```python\n# first code snippet\nx = 2\nprint(x)\n```\n\n```python\n# second code snippet\ny = 6\nprint(x + y)  # This works because state persist\n```\n\nBegin your agentic coding loop!\n\n", hand

In [314]:
result.to_input_list()

[{'content': 'Use an open source weather API to get the temperature in Halifax, Nova Scotia for the last 60 days. Then fit a statistical model to predict the temperature for the next week. Print the final predictions in a markdown table. Todays date is 2025-04-01.',
  'role': 'user'},
 {'id': 'msg_67ec8043d1f88191ab7a95ab93ccdc4508ede4ef04aa1260',
  'content': [{'annotations': [],
    'text': "To solve this problem, we'll follow these steps:\n\n1. **Find an Open Source Weather API**: Identify an API to retrieve past weather data for Halifax, Nova Scotia.\n2. **Fetch the Data**: Gather temperature data for the past 60 days.\n3. **Process Data**: Preprocess the data to fit a statistical model.\n4. **Fit a Model**: Use a statistical model to predict the next week's temperatures.\n5. **Output Predictions**: Print the predictions in a markdown table.\n\nLet's start by finding and using an open source weather API. I'll search for one first.",
    'type': 'output_text'}],
  'role': 'assistant

We could hack together a function to display the conversation like this:

In [315]:
def show_conversation(conversation):
    from ast import literal_eval
    for message in conversation:
        if message.get('role') == 'user':
            show(render_md(f'### User:'))
            show(render_md(message.get('content', '')))
        
        elif message.get('role') == 'assistant':
            show(render_md(f'### Assistant:'))
            # Handle the new content format which is a list of dictionaries
            if isinstance(message.get('content'), list):
                for content_item in message['content']:
                    if content_item.get('type') == 'output_text':
                        show(render_md(content_item.get('text', '')))
        
        # Handle tool calls (like execute_code)
        elif message.get('type') == 'function_call':
            show(render_md(f'### Tool Call: {message.get("name")}'))
            if 'arguments' in message:
                arguments = json.loads(message["arguments"])
                for k, v in arguments.items():
                    show(render_md(f'#### Argument: {k}'))
                    show(render_md(f'```python\n{v}\n```'))
        
        # Handle tool outputs
        elif message.get('type') == 'function_call_output':
            output = message.get("output", {})
            output = literal_eval(output)
            for k, v in output.items():
                if not v:
                    continue
                show(render_md(f'#### Output: {k}'))
                show(render_md(f'```python\n{v}\n```'))

# Use it like this:
show_conversation(result.to_input_list())

Date,Predicted Temperature
2025-04-01,6.14°C
2025-04-02,6.15°C
2025-04-03,6.15°C
2025-04-04,6.15°C
2025-04-05,6.15°C
2025-04-06,6.15°C
2025-04-07,6.15°C


We can display the whole message history with: `result.to_input_list()`

# Resources 

[LangGraph CodeAct](https://www.youtube.com/watch?v=M8E5VekVVss)
